
# PCA aplicado al dataset **Iris** (paso a paso)

Este notebook muestra cómo aplicar **Análisis de Componentes Principales (PCA)** sobre el dataset **Iris**, incluyendo:
- Carga y exploración del dataset.
- Estandarización, ajuste de PCA y **varianza explicada**.
- Visualización en **2D y 3D** en el espacio de PCs.
- **Biplot** con *scores* y *loadings*.
- **Reconstrucción** desde *k* PCs y error de reconstrucción.
- Uso de PCA en un **pipeline** con **Regresión Logística** para clasificación.

**Autor:** DataMining TA‑GPT · **Fecha:** 2025-09-02

> Requisitos: `numpy`, `pandas`, `matplotlib`, `scikit-learn`.


In [ ]:
## 1. Imports y carga del dataset Iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Fijar semilla para reproducibilidad
np.random.seed(42)

# Cargar Iris
iris = load_iris()
X = iris.data  # 150 x 4
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df.head()


In [ ]:
## 2. Exploración rápida
# Tamaño, variables y distribución de clases
print("Forma de X:", X.shape)
print("Variables:", feature_names)
unique, counts = np.unique(y, return_counts=True)
print("Clases y frecuencias:", dict(zip(target_names, counts)))
pd.DataFrame({'clase': target_names, 'frecuencia': counts})

In [ ]:
# Agregar columna con nombres de especies
df['species'] = df['target'].map(dict(enumerate(target_names)))

# Pairplot con seaborn
sns.pairplot(
    df,
    vars=feature_names,      # columnas numéricas
    hue="species",           # color por especie
    diag_kind="hist",        # hist en la diagonal
    plot_kws={"alpha": 0.8, "s": 40, "edgecolor": "white"}
)

plt.suptitle("Matriz de dispersión – Iris", y=1.02)
plt.show()



In [ ]:

## 3. Estandarización y PCA
# Estandarizamos (z-score) y ajustamos PCA
scaler = StandardScaler(with_mean=True, with_std=True)
X_std = scaler.fit_transform(X)

# Ajustar PCA para las 4 PCs
pca = PCA(n_components=4)
scores = pca.fit_transform(X_std)  # proyecciones (scores)
eigvals = pca.explained_variance_
explained_ratio = pca.explained_variance_ratio_
components = pca.components_.T  # loadings: columnas=PCs

print("Varianza explicada por PC:", np.round(explained_ratio, 3))
print("Autovalores (varianza):", np.round(eigvals, 3))
pd.DataFrame(components, index=feature_names, columns=[f'PC{i+1}' for i in range(4)])


In [ ]:

## 4. Scree plot (proporción de varianza explicada)
plt.figure()
plt.plot(np.arange(1, 5), explained_ratio, marker="o")
plt.xticks([1,2,3,4])
plt.xlabel("Componente principal")
plt.ylabel("Proporción de varianza explicada")
plt.title("Scree plot - Iris")
plt.ylim(0, 1)
plt.show()

# Bar Scree plot
plt.figure()
plt.bar(np.arange(1, 5), explained_ratio, color="skyblue")
plt.xticks([1,2,3,4])
plt.ylabel("Proporción de varianza explicada")
plt.xlabel("Componente principal")
plt.title("Bar Scree plot - Iris")
plt.ylim(0,1)
plt.show()


In [ ]:

## 5. Dispersión 2D en PC1-PC2 (por clase)
plt.figure()
for k, name in enumerate(target_names):
    idx = (y == k)
    plt.scatter(scores[idx, 0], scores[idx, 1], label=name)
plt.axhline(0, linestyle="--"); plt.axvline(0, linestyle="--")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Iris en espacio de PCs (PC1 vs PC2)")
plt.legend()
plt.show()


In [ ]:

## 6. Dispersión 3D en PC1-PC2-PC3 (opcional)
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (import side-effect)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for k, name in enumerate(target_names):
    idx = (y == k)
    ax.scatter(scores[idx, 0], scores[idx, 1], scores[idx, 2], label=name)
ax.set_xlabel("PC1"); ax.set_ylabel("PC2"); ax.set_zlabel("PC3")
ax.set_title("Iris en PCs (3D)")
plt.legend()
plt.show()


In [ ]:

## 7. Biplot sencillo en PC1-PC2
loadings = components[:, :2] * np.sqrt(eigvals[:2])

plt.figure()
for k, name in enumerate(target_names):
    idx = (y == k)
    plt.scatter(scores[idx, 0], scores[idx, 1], label=name)

# Vectores de variables (loadings)
for j, feat in enumerate(feature_names):
    plt.arrow(0, 0, loadings[j, 0], loadings[j, 1], head_width=0.1, length_includes_head=True)
    plt.text(loadings[j, 0]*1.1, loadings[j, 1]*1.1, feat)

plt.axhline(0, linestyle="--"); plt.axvline(0, linestyle="--")
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.title("Biplot (scores y loadings) - Iris")
plt.legend(); plt.show()


In [ ]:

## 8. Reconstrucción desde k PCs y error (MSE)
def reconstruct_from_k(scores, components, scaler, k):
    # scores: 150 x 4, components: 4x4 (columnas PCs)
    Vk = components[:, :k]         # 4 x k
    Sk = scores[:, :k]             # 150 x k
    Xk_std = Sk @ Vk.T             # volver al espacio estandarizado
    # Deshacer estandarización
    Xk = scaler.inverse_transform(Xk_std)
    return Xk

mses = []
for k in range(1, 5):
    Xk = reconstruct_from_k(scores, components, scaler, k)
    mse = np.mean((X - Xk) ** 2)
    mses.append(mse)
    print(f"k={k} PCs -> MSE reconstrucción: {mse:.5f}")

plt.figure()
plt.plot(range(1, 5), mses, marker="o")
plt.xlabel("Número de PCs (k)")
plt.ylabel("MSE de reconstrucción")
plt.title("Error de reconstrucción vs k")
plt.show()


In [ ]:

## 9. PCA en pipeline para clasificación
# Comparamos exactitud (accuracy) con y sin PCA mediante validación cruzada.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Modelo base (sin PCA)
clf = LogisticRegression(max_iter=200, multi_class='auto')
acc_base = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
print("Accuracy base (sin PCA):", np.round(acc_base.mean(), 3), "+/-", np.round(acc_base.std(), 3))

# Pipeline con escalado + PCA(2) + LR
pipe_pca2 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('lr', LogisticRegression(max_iter=200, multi_class='auto'))
])
acc_pca2 = cross_val_score(pipe_pca2, X, y, cv=cv, scoring='accuracy')
print("Accuracy con PCA(2):", np.round(acc_pca2.mean(), 3), "+/-", np.round(acc_pca2.std(), 3))

# Pipeline con PCA(3)
pipe_pca3 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=3)),
    ('lr', LogisticRegression(max_iter=200, multi_class='auto'))
])
acc_pca3 = cross_val_score(pipe_pca3, X, y, cv=cv, scoring='accuracy')
print("Accuracy con PCA(3):", np.round(acc_pca3.mean(), 3), "+/-", np.round(acc_pca3.std(), 3))



## 10. Interpretación rápida
- **PC1** suele capturar la mayor variabilidad relacionada con tamaño general del cáliz y pétalo.
- **PC2** separa sutilmente entre clases con combinaciones de ancho/largo de pétalos.
- Con **2–3 PCs** se logra una representación de baja dimensión con **poca pérdida de información**, útil para visualización y *preprocessing* de modelos.
